# GPT4oを用いた画像処理について試す

## 概要

GPT4oを用いて画像処理を行う方法を試し、どのような活用方法が考えられるかを考察する。

## 事前準備

1. 以下公式よりAzureOpenAIリソースを作成し、GPT4oのモデルをデプロイし使用可能な状態とする。
    - [Azure OpenAIリソースの作成](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/how-to/create-resource?pivots=web-portal)
2. .envファイルを作成する。

    ```
    cp .devcontainer/.env.example .devcontainer/.env
    ```

3. .envファイルの下記項目に適切な値を記述する。

    ```
    OPENAI_API_KEY=xxxxxxxx
    OPENAI_API_BASE=https://{resource name}.openai.azure.com/
    OPENAI_API_VERSION=2024-02-01
    OPENAI_DEPLOYMENT_NAME_GPT4o={deployment name}
    ```

4. Devcontainerを起動する。

## Install

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


## Environment

In [2]:
import os

AZURE_OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.environ.get('OPENAI_API_BASE')
DEPLOYMENT_NAME = os.environ.get('OPENAI_DEPLOYMENT_NAME_GPT4o')
API_VERSION = os.environ.get('OPENAI_API_VERSION')

## Usage

参考：[公式サンプルコード](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/gpt-v-quickstart?tabs=image%2Ccommand-line&pivots=programming-language-python#create-a-new-python-application)

In [10]:
from openai import AzureOpenAI

def gptImageAnalysis(userMessage :str, imageUrl :str):
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,  
        api_version=API_VERSION,
        base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}"
    )

    response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=[
            { "role": "system", "content": "You are a helpful assistant. Please output in Japanese." },
            { "role": "user", "content": [  
                { 
                    "type": "text", 
                    "text": userMessage 
                },
                { 
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl
                    }
                }
            ] } 
        ],
        # max_tokens=2000 　GPT4oはデフォルト4096の為指定不要
    )

    return response

def getOutputMessage(response):
    return response.choices[0].message.content

試しに上記の公式サンプルコードにあった画像を使用  
![](https://learn.microsoft.com/ja-jp/azure/ai-services/openai/media/quickstarts/endpoint.png#lightbox)

In [11]:
message = "Describe this picture:"
url = 'https://learn.microsoft.com/ja-jp/azure/ai-services/openai/media/quickstarts/endpoint.png#lightbox'
response = gptImageAnalysis(message, url)

print(getOutputMessage(response))

この画像は、Microsoft Azureポータルの「Keys and Endpoint」の設定画面を示しています。この画面では、ユーザーがCognitive Service APIのアクセスキーとエンドポイントを管理することができます。

画面の主要部分には以下の情報が含まれています：
- 「Regenerate Key1」と「Regenerate Key2」のボタンがあり、キーの再生成が可能です。
- 「Show Keys」ボタンをクリックすると、キーの内容を表示します。
- 「KEY 1」と「KEY 2」にはAPIのアクセスキーが表示され、セキュリティのために隠されています。
- 「Location/Region」セクションでは、「eastus」リージョンが指定されています。
- エンドポイントのURLが「https://docs-test-001.openai.azure.com/」として表示されています。

左側のナビゲーションメニューには以下の項目があります：
- 概要 (Overview)
- アクティビティ ログ (Activity log)
- アクセス制御 (IAM) (Access control (IAM))
- タグ (Tags)
- 問題の診断と解決 (Diagnose and solve problems)
- リソース管理 (Resource Management): この中に「Keys and Endpoint」が強調表示されています。
- その他の項目として、デプロイの管理 (Deployments)、価格設定レベル (Pricing tier)、ネットワーキング (Networking)、ID 管理 (Identity)、コスト分析 (Cost analysis)、プロパティ (Properties)、ロック (Locks) などがあります。

この画面は、ユーザーがAzure Cognitive Servicesのアクセス設定を安全に管理できるように設計されています。


#### URLだけではなく、画像ファイルをBase64に変換し処理することも可能

In [22]:
import urllib.request
import base64

def convertImageURLtoBase64data(url):
    # 画像ファイルダウンロード
    dst_path = '../data/temp/temp.png'
    urllib.request.urlretrieve(url, dst_path)

    # base64 エンコード
    with open(dst_path, 'rb') as image:
        base64_data = base64.b64encode(image.read()).decode('utf-8')

    return f"data:image/jpg;base64,{base64_data}"


In [23]:
url = 'https://learn.microsoft.com/ja-jp/azure/ai-services/openai/media/quickstarts/endpoint.png#lightbox'

base64_image_data = convertImageURLtoBase64data(url)
response = gptImageAnalysis(message, base64_image_data)

print(getOutputMessage(response))

この画像はMicrosoft Azureのポータル画面を示しています。特に「docs-test-001」というCognitive Serviceリソースの「Keys and Endpoint」（鍵とエンドポイント）の設定画面です。

- 画面の左側にはナビゲーションメニューがあり、「Resource Management」（リソース管理）の一部として「Keys and Endpoint」が選択されています。
- 右側のメインコンテンツには、APIキーとエンドポイントの情報が表示されています。
  - 「KEY 1」と「KEY 2」の2つのキーが表示されていますが、実際のキーの内容は隠されています。
  - 「Location/Region」（場所/地域）は「eastus」に設定されています。
  - エンドポイントは「https://docs-test-001.openai.azure.com/」と記載されています。
- また、画面の上部には「Regenerate Key1」と「Regenerate Key2」というキーの再生成オプションがあります。
- 情報ボックスには、これらのキーがCognitive Service APIにアクセスするために使用される旨、およびキーのセキュリティと定期的な再生成の推奨事項が記載されています。

この設定画面は、Azure Cognitive ServiceのAPIエンドポイントやアクセスキーを管理するためのものです。


## Example

### 01.グラフ情報の読み取り